In [1]:
# Import Python package
import BinhoSupernova
from BinhoSupernova.Supernova import Supernova
from BinhoSupernova.commands.definitions import *

## Getting started

#### 1. List all the Supernova devices connected to the PC host

The ``BinhoSupernova.getConnectedSupernovaDevicesList()`` gets a list of the Supernova devices plugged into the host PC machine.

In [2]:
BinhoSupernova.getConnectedSupernovaDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FC#6&3ac173e5&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fc',
  'serial_number': '7742E9E56C791D5C9A54FCCC961E897',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Supernova',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Supernova class

To utilize a Supernova USB host adapter device, we need to create an instance of the Supernova class.

In [3]:
# Create a device instance. One instance of the Supernova class represents a Supernova USB host adapter device.
supernova = Supernova()

#### 3. Open connection to the Supernova device

The public method ``Supernova.open()`` establishes the connection with a Supernova device. Below is the complete signature:

```python
open(vid, pid, serial, path)
```

- ``vid: int``: The Supernova USB VID, which is set by default.
- ``pid: int``: The Supernova USB PID, which is set by default.
- ``serial: str``: The Supernova serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoSupernova.getConnectedSupernovaDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Supernova device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Supernova device simultaneously.

In [4]:
# Use the method by default to connect to only one Supernova device.
supernova.open()

# Otherwise, use the path attribute to identify each Supernova device. Uncomment the line below and comment the line #2.
# supernova.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Supernova device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Supernova

To handle responses and notifications from Supernova, a callback function must be defined and registered. This function will be invoked every time the Supernova sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(supernova_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Supernova.onEvent(callback_function)`` method.

In [5]:
import json

# Define callback function
def callback_function(supernova_message: dict, system_message: dict) -> None:
    
    if supernova_message != None:

        # Print a header
        print(">> New message from SUPERNOVA:")

        # If the command is GET TARGET DEVICE TABLE, convert to json.
        if supernova_message["name"] == "I3C GET TARGET DEVICE TABLE" or supernova_message["name"] == "I3C TRANSFER":
            responseJson = json.dumps(supernova_message, indent=4)
            print(responseJson)

        else:
            print(supernova_message)
    
    if system_message != None:

        # Print a header
        print(">> New message from the SYSTEM:")

        print(system_message)

In [6]:
# Register callback function
supernova.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Supernova from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[0, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [7]:
#Auxiliar code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

## I3C Protocol API

### 1. Configuring the supernova/PIC18QF16Q20/I2C FRAM Setup

To facilitate the Protocol Translator test:

**Necessary Firmware:**
- Supernova firmware version 2.0.0
- Supernova SDK version 2.0.0
- Custom image for the PIC18QF16Q20 protocol translator (located in the application folder)

**Connection Arrangement:**

<img src="assets/block_diagram.png" alt="connection" width="60%">

<img src="assets/connection.png" alt="connection_diagram" width="40%">

- Link the supernova High voltage I3C tiger eye to Qwiic connector with the I3C Qwiic connector on the Supernova PIC18QF16Q20 board.
- Establish connections for SDA and SCL from the I2C/I3C Qwiic connector on the Supernova PIC18QF16Q20 board to the corresponding SCL/SDA channels of the I2C FRAM.
- Connect VCC and GND from the I2C Qwiic on the Supernova to the VCC and GND of the I2C FRAM.

   ![Connection Image Placeholder]

**Testing Procedure:**

The objective of this test is to utilize the PIC18QF16Q20 as a protocol translator, issuing custom I3C commands to execute the following transactions:
- I2C write


<img src="assets/i2c_write.png" alt="connection" width="80%">

- I2C read

![I2C_read](assets/i2c_read.png)

- SPI write

![SPI_write](assets/spi_write.png)

- SPI read

![SPI_read](assets/spi_read.png)

- SPI write followed by read

<img src="assets/spi_write_read.png" alt="connection" width="70%">


For I2C transactions, an I2C FRAM will be employed for data read and write operations. To assess SPI functionality, the Supernova PIC18QF16Q20 board integrates a W25Q64JV (64M-bit) SPI Serial Flash, allowing for the execution of commands and read/write operations.

### 2. Initialize the Supernova I3C peripheral as controller 

In [8]:
supernova.i3cControllerInit(getId())

{'module': 1,
 'opcode': 0,
 'message': 'INITIALIZE I3C CONTROLLER requests success'}

>> New message from SUPERNOVA:
{'id': 1, 'command': 1, 'name': 'I3C CONTROLLER INIT', 'result': 'I3C_CONTROLLER_INIT_SUCCESS'}


### 3. Adjust I3C Bus Voltage

As this example utilizes the secondary I3C bus on the PIC18QF16Q20, it is essential to set the bus voltage to 1.2V for optimal operation.

In [9]:
request_result = supernova.setI3cBusVoltage(getId(), 1200)

>> New message from SUPERNOVA:
{'id': 2, 'command': 97, 'name': 'SET I3C BUS VOLTAGE', 'result': 0}


### 4. Initialize the I3C Bus

1. Examine the device table prior to the PIC18QF16Q20 hot-joining.

In [10]:
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 3,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 0,
    "table": []
}


2. I3C Bus Initialization

The Supernova conducts an I3C initialization on an unpopulated bus, resulting in no target ACKs. Following the execution of this cell, connect the PIC18QF16Q20. The PIC18QF16Q20 firmware performs a hot-join, integrating itself into the I3C bus and updating the device table.

In [11]:
# Initialize and scan the I3C bus.
request_result = supernova.i3cInitBus(getId())

>> New message from SUPERNOVA:
{'id': 4, 'command': 2, 'name': 'I3C INIT BUS', 'result': 'RSTDAA_FAILED', 'errors': ['NACK_ERROR']}


>> New message from SUPERNOVA:
{'id': 4, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_HOT_JOIN', 'response': 'IBI_ACKED', 'hasData': True, 'length': 8, 'status': 0}, 'pid': ['0x06', '0x9a', '0x00', '0x00', '0x00', '0x00'], 'bcr': 30, 'dcr': 198}


3. Examine the device table after the PIC18QF16Q20 hot-joining.

In [12]:
request_result = supernova.i3cGetTargetDeviceTable(getId())

>> New message from SUPERNOVA:
{
    "id": 5,
    "command": 3,
    "name": "I3C GET TARGET DEVICE TABLE",
    "numberOfTargets": 1,
    "table": [
        {
            "staticAddress": 0,
            "dynamicAddress": 8,
            "bcr": {
                "value": [
                    "0b00011110",
                    30,
                    "0x1e"
                ],
                "description": {
                    "deviceRole": "I3C Target.",
                    "advancedCapabilities": "Does not support optional advanced capabilities.",
                    "virtualTargetSupport": "Is a Virtual Target, or exposes other downstream Device(s).",
                    "offlineCapable": "Device will not always respond to I3C Bus commands.",
                    "ibiPayload": "One data byte (MDB) shall follow the accepted IBI, and additional data bytes may follow.",
                    "ibiRequestCapable": "Capable.",
                    "maxDataSpeedLimitation": "No Limitation."
     

### 5. Communicate with the I2C peripheral

To verify the proper functionality of the I2C FRAM, we will execute the following transactions:

1. **I2C Write Transaction:**
   - Write data [0x06,0x07,0x08,0x09,0x0A] to the register [0x00,0x00].

2. **I2C Write to Specific Register:**
   - Direct an I2C write to the register [0x00,0x00].

3. **I2C Read Transaction:**
   - Perform an I2C read of length 5 to retrieve the previously written data.

#### Execution Steps:
To initiate the first transaction, execute an I3C write to the dynamic address of the PIC18QF16Q20 (0x08). The data to be written includes:
   - Command ID for the I2C write command (0x40)
   - I2C address right-shifted by one (original address: 0x50, shifted address: 0xA0)
   - Register to write to [0x00,0x00]
   - Data to be written [0x06,0x07,0x08,0x09,0x0A]

In [13]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = []
DATA            = [0x40, 0xA0,0x00, 0x00, 0x06, 0x07, 0x08, 0x09, 0x0A]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 6,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


For the second transaction, resetting the data pointer to the [0x00,0x00] register is achieved by performing an I3C write with the following data:

   - Command ID for the I2C write command (0x40)
   - I2C address right-shifted by one (original address: 0x50, shifted address: 0xA0)
   - Register to write to [0x00,0x00]
   - Empty data payload
 

In [14]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = []
DATA            = [0x40, 0xA0, 0x00, 0x00]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 7,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


For the final transaction, executing a write command involves sending the following I3C data:

   - Command ID for the I2C read command (0x20)
   - I2C address right-shifted by one (original address: 0x50, shifted address: 0xA0)
   - Amount of data to read (0x05)

This initiates an I2C read to the designated address with the specified length. The data resulting from this read will be received via I3C in an IBI with payload. This IBI includes a mandatory data byte of 0x00, and the remaining payload constituting the read data.

In [15]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = []
DATA            = [0x20, 0xA0, 0x05]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 8,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 6, 'status': 0}, 'MDB': {'value': 0, 'description': {'category': 'User defined.', 'description': 'Defined by the vendor and reserved for vendor definition.'}}, 'payload': [0, 6, 7, 8, 9, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


### 6. Communicate with the SPI peripheral

To validate the SPI communication, we will execute specific SPI commands tailored for the Supernova PIC18QF16Q20 board's SPI FLASH. Here are some examples:

1. **Get JEDEC ID:**
   - Execute the command to retrieve the JEDEC ID from the SPI FLASH.

2. **Write Status Register-1:**
   - Perform the command to write to Status Register-1.

3. **Read Status Register-1:**
   - Execute the command to read from Status Register-1.

These commands will allow us to assess the SPI communication and validate the Supernova PIC18QF16Q20 board's interaction with the SPI FLASH.

To obtain the JEDEC ID, the SPI command 0x9F should be issued via SPI, followed by reading 3 bytes of data. To accomplish this, the write/read SPI command (0x60) for the PIC18QF16Q20 should be utilized, requiring the following data to be sent via I3C:

- Command ID for the SPI write/read command (0x60)
- Amount of data to read (0x03)
- Data to write (the SPI command for getting JEDEC ID 0x9F)

Similar to the I2C read transaction, once the data is retrieved via SPI, an IBI will be sent via I3C with the mandatory data byte (MDB) set to 0x00 and the 3 bytes of read data.

In [16]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = []
DATA            = [0x60, 0x03, 0x9F]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 9,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 4, 'status': 0}, 'MDB': {'value': 0, 'description': {'category': 'User defined.', 'description': 'Defined by the vendor and reserved for vendor definition.'}}, 'payload': [0, 239, 64, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


To perform the Write Status Register-1 SPI command, we need to first enable write transactions by issuing the Write Enable command (0x06). The necessary data to be sent via I3C includes:

- Command ID for the SPI write command (0x41)
- Data to write (the SPI Write Enable command value 0x9F)




In [17]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = []
DATA            = [0x41, 0x06]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 10,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


To execute the Write Status Register-1 SPI command, the requisite data to be sent via I3C includes:

- Command ID for the SPI write command (0x41)
- Data to write
    - Write Status Register-1 SPI command (0x01)
    - Data to be written to the register (0x60)

Configuring the Register-1 with the value 0x60 involves setting:



![I2C_read](assets/Register.png)

- SEC to 1: Block Protect Bits (BP2, BP1, BP0) protect 4KB Sectors.
- TB to 1: Block Protect Bits (BP2, BP1, BP0) protect from the Bottom.






In [18]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = []
DATA            = [0x41, 0x01, 0x60]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 11,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


To execute the Read Status Register-1 SPI command, the following I3C data must be transmitted:

- Command ID for the SPI write/read command (0x60)
- Amount of data to read (0x01 from the register) 
- Data to write (SPI command for reading the Status Register-1, 0x05)

Similar to the previous transactions, the data from the Status Register-1 value is transmitted via IBI on the I3C bus. As configured earlier, the expected value should be 0x60.

In [19]:
TARGET_ADDR     = 0x08
MODE            = TransferMode.I3C_SDR
PUSH_PULL_RATE  = I3cPushPullTransferRate.PUSH_PULL_5_MHZ
OPEN_DRAIN_RATE = I3cOpenDrainTransferRate.OPEN_DRAIN_500_KHZ
REG_ADDR        = []
DATA            = [0x60, 0x01, 0x05]        # Python list of bytes. Maximum length of 1024 bytes.

# API I3C Write
request_result = supernova.i3cWrite(getId(), 
                                    TARGET_ADDR, 
                                    MODE, 
                                    PUSH_PULL_RATE, 
                                    OPEN_DRAIN_RATE, 
                                    REG_ADDR, 
                                    DATA)

>> New message from SUPERNOVA:
{
    "id": 12,
    "command": 11,
    "name": "I3C TRANSFER",
    "header": {
        "tag": "RESPONSE_TO_REGULAR_REQUEST",
        "result": "I3C_TRANSFER_SUCCESS",
        "hasData": false
    },
    "descriptor": {
        "dataLength": 0,
        "errors": [
            "NO_TRANSFER_ERROR"
        ]
    },
    "data": []
}


>> New message from SUPERNOVA:
{'id': 0, 'command': 12, 'name': 'I3C IBI NOTIFICATION', 'header': {'tag': 'INTERRUPT_NOTIFICATION', 'address': 8, 'type': 'IBI_NORMAL', 'response': 'IBI_ACKED_WITH_PAYLOAD', 'hasData': True, 'length': 2, 'status': 0}, 'MDB': {'value': 0, 'description': {'category': 'User defined.', 'description': 'Defined by the vendor and reserved for vendor definition.'}}, 'payload': [0, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
